In [1]:
import pandas as pd
import numpy as np

# Load dataset
df = pd.read_csv("../data/Creditcard_data.csv")

# Basic check
df.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,1
2,1,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [2]:
# Dataset dimensions
print("Dataset shape:", df.shape)

# Class distribution
class_counts = df['Class'].value_counts()
print("\nClass distribution:")
print(class_counts)

# Percentage distribution
print("\nClass percentage:")
print(class_counts / len(df) * 100)

Dataset shape: (772, 31)

Class distribution:
Class
0    763
1      9
Name: count, dtype: int64

Class percentage:
Class
0    98.834197
1     1.165803
Name: count, dtype: float64


In [3]:
# Separate features and target
X = df.drop('Class', axis=1)
y = df['Class']

print("Feature matrix shape:", X.shape)
print("Target vector shape:", y.shape)

Feature matrix shape: (772, 30)
Target vector shape: (772,)


In [4]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42,
    stratify=y
)

print("Train class distribution:")
print(y_train.value_counts())

print("\nTest class distribution:")
print(y_test.value_counts())

Train class distribution:
Class
0    610
1      7
Name: count, dtype: int64

Test class distribution:
Class
0    153
1      2
Name: count, dtype: int64


In [5]:
# Sampling libraries
from imblearn.over_sampling import RandomOverSampler, SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.combine import SMOTETomek, SMOTEENN

In [6]:
print("Sampling libraries imported successfully")

Sampling libraries imported successfully


In [7]:
# Sampling 1: Random Oversampling
ros = RandomOverSampler(random_state=42)

X_ros, y_ros = ros.fit_resample(X_train, y_train)

print("After Random Oversampling:")
print(y_ros.value_counts())

After Random Oversampling:
Class
0    610
1    610
Name: count, dtype: int64


In [9]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

# Pipeline: Scaling + Logistic Regression
m1_pipeline = Pipeline([
    ("scaler", StandardScaler()),
    ("lr", LogisticRegression(max_iter=3000))
])

# Train on oversampled data
m1_pipeline.fit(X_ros, y_ros)

# Predict on original test set
y_pred_m1_s1 = m1_pipeline.predict(X_test)

# Accuracy
acc_m1_s1 = accuracy_score(y_test, y_pred_m1_s1)

print("M1 (Logistic Regression) accuracy with Sampling1:", round(acc_m1_s1 * 100, 2), "%")

M1 (Logistic Regression) accuracy with Sampling1: 90.97 %


In [10]:
# Sampling 2: SMOTE
smote = SMOTE(random_state=42)

X_smote, y_smote = smote.fit_resample(X_train, y_train)

print("After SMOTE:")
print(y_smote.value_counts())

After SMOTE:
Class
0    610
1    610
Name: count, dtype: int64


In [11]:
# Train Logistic Regression (with scaling) on SMOTE data
m1_pipeline.fit(X_smote, y_smote)

# Predict on original test set
y_pred_m1_s2 = m1_pipeline.predict(X_test)

# Accuracy
acc_m1_s2 = accuracy_score(y_test, y_pred_m1_s2)

print("M1 (Logistic Regression) accuracy with Sampling2 (SMOTE):", round(acc_m1_s2 * 100, 2), "%")

M1 (Logistic Regression) accuracy with Sampling2 (SMOTE): 91.61 %


In [12]:
# Sampling 3: Random Undersampling
rus = RandomUnderSampler(random_state=42)

X_rus, y_rus = rus.fit_resample(X_train, y_train)

print("After Random Undersampling:")
print(y_rus.value_counts())

After Random Undersampling:
Class
0    7
1    7
Name: count, dtype: int64


In [13]:
# Train Logistic Regression (with scaling) on undersampled data
m1_pipeline.fit(X_rus, y_rus)

# Predict on original test set
y_pred_m1_s3 = m1_pipeline.predict(X_test)

# Accuracy
acc_m1_s3 = accuracy_score(y_test, y_pred_m1_s3)

print("M1 (Logistic Regression) accuracy with Sampling3 (Random Undersampling):",
      round(acc_m1_s3 * 100, 2), "%")

M1 (Logistic Regression) accuracy with Sampling3 (Random Undersampling): 58.06 %


In [14]:
# Sampling 4: SMOTE + Tomek Links
smote_tomek = SMOTETomek(random_state=42)

X_smt, y_smt = smote_tomek.fit_resample(X_train, y_train)

print("After SMOTETomek:")
print(y_smt.value_counts())

After SMOTETomek:
Class
0    590
1    590
Name: count, dtype: int64


In [15]:
# Train Logistic Regression (with scaling) on SMOTETomek data
m1_pipeline.fit(X_smt, y_smt)

# Predict on original test set
y_pred_m1_s4 = m1_pipeline.predict(X_test)

# Accuracy
acc_m1_s4 = accuracy_score(y_test, y_pred_m1_s4)

print("M1 (Logistic Regression) accuracy with Sampling4 (SMOTETomek):",
      round(acc_m1_s4 * 100, 2), "%")

M1 (Logistic Regression) accuracy with Sampling4 (SMOTETomek): 91.61 %


In [16]:
# Sampling 5: SMOTE + Edited Nearest Neighbours
smote_enn = SMOTEENN(random_state=42)

X_senn, y_senn = smote_enn.fit_resample(X_train, y_train)

print("After SMOTEENN:")
print(y_senn.value_counts())

After SMOTEENN:
Class
1    474
0    379
Name: count, dtype: int64


In [17]:
# Train Logistic Regression (with scaling) on SMOTEENN data
m1_pipeline.fit(X_senn, y_senn)

# Predict on original test set
y_pred_m1_s5 = m1_pipeline.predict(X_test)

# Accuracy
acc_m1_s5 = accuracy_score(y_test, y_pred_m1_s5)

print("M1 (Logistic Regression) accuracy with Sampling5 (SMOTEENN):",
      round(acc_m1_s5 * 100, 2), "%")

M1 (Logistic Regression) accuracy with Sampling5 (SMOTEENN): 90.32 %


In [18]:
from sklearn.neighbors import KNeighborsClassifier

# KNN pipeline (scaling is mandatory)
m2_pipeline = Pipeline([
    ("scaler", StandardScaler()),
    ("knn", KNeighborsClassifier(n_neighbors=5))
])

m2_results = {}

# Sampling datasets dictionary
sampling_data = {
    "Sampling1": (X_ros, y_ros),
    "Sampling2": (X_smote, y_smote),
    "Sampling3": (X_rus, y_rus),
    "Sampling4": (X_smt, y_smt),
    "Sampling5": (X_senn, y_senn)
}

for samp_name, (X_samp, y_samp) in sampling_data.items():
    m2_pipeline.fit(X_samp, y_samp)
    y_pred = m2_pipeline.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    m2_results[samp_name] = round(acc * 100, 2)

print("M2 (KNN) results:")
m2_results

M2 (KNN) results:


{'Sampling1': 96.13,
 'Sampling2': 94.19,
 'Sampling3': 65.81,
 'Sampling4': 94.19,
 'Sampling5': 91.61}

In [19]:
from sklearn.tree import DecisionTreeClassifier

m3 = DecisionTreeClassifier(random_state=42)

m3_results = {}

for samp_name, (X_samp, y_samp) in sampling_data.items():
    m3.fit(X_samp, y_samp)
    y_pred = m3.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    m3_results[samp_name] = round(acc * 100, 2)

print("M3 (Decision Tree) results:")
m3_results

M3 (Decision Tree) results:


{'Sampling1': 99.35,
 'Sampling2': 96.13,
 'Sampling3': 50.97,
 'Sampling4': 96.77,
 'Sampling5': 87.74}

In [20]:
from sklearn.ensemble import RandomForestClassifier

m4 = RandomForestClassifier(
    n_estimators=200,
    random_state=42,
    n_jobs=-1
)

m4_results = {}

for samp_name, (X_samp, y_samp) in sampling_data.items():
    m4.fit(X_samp, y_samp)
    y_pred = m4.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    m4_results[samp_name] = round(acc * 100, 2)

print("M4 (Random Forest) results:")
m4_results

M4 (Random Forest) results:


{'Sampling1': 99.35,
 'Sampling2': 99.35,
 'Sampling3': 63.87,
 'Sampling4': 99.35,
 'Sampling5': 98.71}

In [21]:
from sklearn.svm import SVC

# SVM pipeline with scaling
m5_pipeline = Pipeline([
    ("scaler", StandardScaler()),
    ("svm", SVC(kernel="rbf", gamma="scale"))
])

m5_results = {}

for samp_name, (X_samp, y_samp) in sampling_data.items():
    m5_pipeline.fit(X_samp, y_samp)
    y_pred = m5_pipeline.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    m5_results[samp_name] = round(acc * 100, 2)

print("M5 (SVM) results:")
m5_results

M5 (SVM) results:


{'Sampling1': 94.84,
 'Sampling2': 95.48,
 'Sampling3': 41.94,
 'Sampling4': 95.48,
 'Sampling5': 94.19}

In [22]:
# Combine all results into one table
final_results = pd.DataFrame({
    "Sampling1": {
        "M1": 90.97,
        "M2": m2_results["Sampling1"],
        "M3": m3_results["Sampling1"],
        "M4": m4_results["Sampling1"],
        "M5": m5_results["Sampling1"]
    },
    "Sampling2": {
        "M1": 91.61,
        "M2": m2_results["Sampling2"],
        "M3": m3_results["Sampling2"],
        "M4": m4_results["Sampling2"],
        "M5": m5_results["Sampling2"]
    },
    "Sampling3": {
        "M1": 58.06,
        "M2": m2_results["Sampling3"],
        "M3": m3_results["Sampling3"],
        "M4": m4_results["Sampling3"],
        "M5": m5_results["Sampling3"]
    },
    "Sampling4": {
        "M1": 91.61,
        "M2": m2_results["Sampling4"],
        "M3": m3_results["Sampling4"],
        "M4": m4_results["Sampling4"],
        "M5": m5_results["Sampling4"]
    },
    "Sampling5": {
        "M1": 90.32,
        "M2": m2_results["Sampling5"],
        "M3": m3_results["Sampling5"],
        "M4": m4_results["Sampling5"],
        "M5": m5_results["Sampling5"]
    }
})

final_results

,Sampling1,Sampling2,Sampling3,Sampling4,Sampling5
M1,90.97,91.61,58.06,91.61,90.32
M2,96.13,94.19,65.81,94.19,91.61
M3,99.35,96.13,50.97,96.77,87.74
M4,99.35,99.35,63.87,99.35,98.71
M5,94.84,95.48,41.94,95.48,94.19


In [23]:
final_results.to_csv("../results/accuracy_table.csv")
print("Results saved to results/accuracy_table.csv")

Results saved to results/accuracy_table.csv


In [24]:
final_tally = final_results.copy()
final_tally.insert(0, "Model", final_tally.index)
final_tally.insert(0, "S.No", range(1, len(final_tally) + 1))
final_tally.reset_index(drop=True, inplace=True)

final_tally

,S.No,Model,Sampling1,Sampling2,Sampling3,Sampling4,Sampling5
0,1,M1,90.97,91.61,58.06,91.61,90.32
1,2,M2,96.13,94.19,65.81,94.19,91.61
2,3,M3,99.35,96.13,50.97,96.77,87.74
3,4,M4,99.35,99.35,63.87,99.35,98.71
4,5,M5,94.84,95.48,41.94,95.48,94.19


In [25]:
final_tally.to_csv("../results/final_tally.csv", index=False)